<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/twadr_1_mixed_pubmed_bigtweet_twadr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [10]:
# training data set

train_csv = path/'TwADR-L.fold-1.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"TwADR-L.fold-1.validation.csv",header=None)
valid.head()

,0,1
0,85,tremor
1,119,anxiety
2,154,hives
3,189,dizziness
4,189,dizziness


In [12]:
# testing data set

test = pd.read_csv(path/"TwADR-L.fold-1.test.csv",header=None)
test.head()

,0,1
0,13,Insomnia
1,13,awake in the middle of the night
2,61,very stupid
3,133,night terrors
4,189,dizziness


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'mixed_pubmed_bigtweet_twadr.csv')

In [14]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'the',
 'of',
 'and',
 'in',
 'to',
 'a',
 'for',
 'with',
 'was',
 'diclofenac',
 'were',
 'on',
 'is',
 'rt',
 'pain',
 'sodium',
 'by',
 'drug',
 'that',
 'as',
 'i',
 'at',
 'or',
 'patients',
 'this',
 'from',
 'you',
 'disease',
 'mg',
 'increased',
 'be',
 'study',
 'group',
 'my',
 'are',
 'mood',
 'not',
 'treatment',
 'after',
 'it',
 'an',
 'have',
 'me',
 'amp',
 'p',
 'gluten',
 'release',
 ',',
 'disorder',
 'all',
 'free',
 'effect',
 'abnormal',
 'we',
 'than',
 'no',
 'withdrawal',
 'symptoms',
 'decreased',
 'compared',
 'out',
 'more',
 'your',
 'using',
 'anxiety',
 'effects',
 'new',
 'weight',
 'day',
 'up',
 'groups',
 'two',
 'our',
 'time',
 'inflammatory',
 'about',
 'drugs',
 'between',
 'its',
 'both',
 'significantly',
 'blood',
 'significant',
 'like',
 'h',
 'anti',
 'had',
 'acute',
 'but',
 'potassium',
 'these',
 'skin',
 'use',
 'disorders',
 'can',
 'which',
 '

In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.748611,4.191018,0.345089,22:53


In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.011661,3.813244,0.381223,22:51


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.728069,3.770616,0.385958,22:52


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.671226,3.737394,0.389759,23:05


In [25]:
learn.save_encoder('pubmed_cadec_twadrl_mixed_fold1_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('pubmed_cadec_twadrl_mixed_fold1_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,7.164419,6.193729,0.114035,00:23


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,6.179261,5.153201,0.131579,00:27


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.690652,4.478945,0.184211,01:10


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.313859,4.183500,0.263158,01:08
1,5.130180,4.040364,0.254386,01:10


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.947289,3.885593,0.271930,01:07
1,4.746967,3.804395,0.280702,01:05


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.472835,3.617919,0.271930,01:11
1,4.309844,3.580219,0.307018,01:15


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.143432,3.443631,0.324561,01:08
1,3.991961,3.464662,0.307018,01:07


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.795244,3.286752,0.333333,01:07
1,3.657757,3.246342,0.315789,01:09


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.519202,3.208546,0.342105,01:09
1,3.419823,3.195864,0.333333,01:10
2,3.241951,3.074203,0.368421,01:11
3,3.058040,3.059177,0.359649,01:15


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.903646,3.041420,0.359649,01:11
1,2.861588,3.037502,0.385965,01:09
2,2.689419,3.015268,0.394737,01:07
3,2.506493,3.000467,0.403509,01:08


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.360404,3.001726,0.412281,01:08
1,2.319497,3.091278,0.385965,01:10
2,2.155585,3.063464,0.438596,01:05
3,1.990394,3.024331,0.429825,01:06


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.835489,3.113526,0.421053,01:08
1,1.806816,3.037092,0.429825,01:10
2,1.668866,2.992236,0.412281,01:11
3,1.551970,2.966848,0.412281,01:10


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.401552,3.004032,0.429825,01:07
1,1.437088,3.141397,0.438596,01:08
2,1.321278,3.044457,0.429825,01:06
3,1.224412,3.076880,0.429825,01:08


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.091559,3.194258,0.438596,01:08
1,1.150246,3.235505,0.429825,01:09
2,1.046751,3.183353,0.447368,01:09
3,0.962639,3.239929,0.438596,01:09


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.860781,3.190727,0.412281,01:09
1,0.915171,3.309189,0.447368,01:09
2,0.853939,3.340638,0.447368,01:05
3,0.769027,3.339736,0.447368,01:07


In [41]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.742477,3.405522,0.464912,01:05
1,0.725380,3.447947,0.447368,01:10


In [42]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.665480,3.304940,0.421053,01:08
1,0.653433,3.404199,0.438596,01:07


In [43]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.629951,3.532238,0.421053,01:11


In [48]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.579087,3.527045,0.429825,01:11


In [44]:
# save the best model

learn.save_encoder('pubmed_cadec_twadrl_mixed_fold1')

# Part three: Predict on the test dataset

In [49]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [50]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

143
143
[1787, 233, 2149, 133, 189, 189, 974, 2147, 1936, 2147, 393, 2032, 1787, 1136, 418, 452, 1610, 744, 749, 97, 1769, 1597, 1458, 2018, 1599, 2074, 824, 825, 1787, 852, 2149, 974, 974, 792, 818, 974, 974, 982, 978, 1942, 982, 978, 1062, 1062, 1669, 1089, 1096, 1787, 1175, 1432, 2113, 1546, 1727, 1295, 852, 1711, 1295, 1545, 1295, 417, 1319, 1597, 755, 1432, 1432, 1483, 1529, 91, 1062, 1545, 2097, 966, 1163, 1597, 1769, 1597, 1599, 392, 1610, 1610, 1610, 1526, 2194, 1610, 1073, 1192, 109, 1531, 1573, 1718, 1597, 1767, 1767, 1798, 1767, 1718, 1597, 2113, 1192, 1787, 2032, 1787, 1799, 2023, 923, 978, 1099, 1109, 1099, 1925, 1936, 1936, 1936, 1340, 1936, 1597, 852, 1822, 852, 1356, 1727, 1813, 1727, 2032, 2149, 1787, 189, 189, 1597, 426, 2081, 2147, 2147, 1099, 2147, 2032, 2147, 1545, 2149, 824, 2191, 1529, 2194]
[13, 13, 61, 133, 189, 189, 195, 233, 233, 233, 393, 393, 393, 396, 418, 452, 561, 744, 749, 750, 791, 791, 809, 813, 813, 818, 824, 824, 825, 852, 947, 974, 974, 974, 974, 9

In [51]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

53
0.3706293706293706
